In [1]:
from neo4j import GraphDatabase

URI = "neo4j+s://fb99990e.databases.neo4j.io"
AUTH = ("neo4j", "lh1LhGuDXcqbw3e4-F4DIN9iFoQLbXY3fiMotLoG9Ac")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [95]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

entity_nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "Where is achham located "

ner_results = entity_nlp(example)
print(ner_results)

/home/prayash/.local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'LOC', 'score': 0.5823375, 'word': 'achham', 'start': 9, 'end': 15}]


In [96]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [97]:
def Pre_Process(query):
    query = query.lower()
    tokens = []
    doc = nlp(query)
    for token in doc:
        tokens.append(token)
    return tokens

Pre_Process("hi my name is Prayash thapa")


[hi, my, name, is, prayash, thapa]

In [98]:
def Entities(query):
    doc = entity_nlp(query.lower())
   #
    return doc

In [99]:
def checkTokenin(token , text):
    for tex in text:
        if token == tex.orth_:
            return True
    
    return False

In [100]:
def AnswerQuery(query):
    text = Pre_Process(query)
    #print(text)
    
    
    
    if checkTokenin("located", text):
        #query the graph for relation located_in_administrative

        #print(Entities(query)[0]['word'])
        name = Entities(query)[0]['word']

        records, summary, keys = driver.execute_query(
        """MATCH (E:Entity{name:$name})-[:_located_in_the_administrative_territorial_entity]->(entity:Entity)
            RETURN entity.name
        """,
        name = name,
        database_="neo4j")

        if len(records) == 0:
            print("Sorry I don't know about this location")
            return
        else:

            print("It is located in " +records[0].value())

    else:
        print("This is beyond my capabilities I only only know where someplace is located")

    #print("do")

In [105]:
AnswerQuery("where is kailali located")


Sorry I don't know about this location
